## Single Gnowee

This scipt is the bare minimum needed to run Gnowee.  

Different objective functions can be added to the ObjectiveFunctions module by following the examples shown.

In [ ]:
# Gnowee Modules
import Gnowee
from ObjectiveFunction import ObjectiveFunction
from GnoweeUtilities import ProblemParameters
from GnoweeHeuristics import GnoweeHeuristics

# Select optimization problem type and associated parameters
gh = GnoweeHeuristics()
gh.set_preset_params('mi_chemical_process', 'Gnowee')

# Change any desired program settings
#gh.initSampling = 'nolh'
print gh

# Run optimization
(timeline) = Gnowee.main(gh)
print '\nThe result:\n', timeline[-1]

## Run Single Gnowee on TSP

In [ ]:
import os

# Gnowee Modules
import Gnowee
from ObjectiveFunction import ObjectiveFunction
from GnoweeUtilities import ProblemParameters
from GnoweeHeuristics import GnoweeHeuristics
from TSP import TSP

# Select optimization problem type and associated parameters
gh = GnoweeHeuristics()
tspProb = TSP()
tspProb.read_tsp(os.path.pardir+'/Benchmarks/TSPLIB/eil51.tsp')
tspProb.build_prob_params(gh)
gh.maxGens = 750
gh.stallLimit = 15000
print gh

# Run optimization
(timeline) = Gnowee.main(gh)
print '\nThe result:\n', timeline[-1]

In [ ]:
from ObjectiveFunction import ObjectiveFunction
from Constraints import Constraint

optFunc = ObjectiveFunction(method='mi_pressure_vessel')
print optFunc.func([38.77342202, 223.7237953, 0.75, 0.375])

conFunc = Constraint(method='mi_pressure_vessel', constraint=0.0)
print conFunc.func([38.77342202, 223.7237953, 0.75, 0.375])

## Batch Gnowee

This scipt runs multipe instances of Gnowee against an optimization prolem.  It is useful for benchmarking and understanding the distribution of solutions obtained by Gnowee for a particular problem.  Statistics and plots are obtained and shown following the optimization process.  

Different objective functions can be added to the ObjectiveFunctions module by following the examples shown.

In [ ]:
import numpy as np
import math as m

from operator import attrgetter

# Gnowee Modules
import Gnowee
import OptiPlot as op
from ObjectiveFunction import ObjectiveFunction
from GnoweeHeuristics import GnoweeHeuristics
from GnoweeUtilities import Event, ProblemParameters

# Select optimization problem type and associated parameters
#problem = 'welded_beam'
#problem = 'pressure_vessel'
#problem = 'speed_reducer'
#problem = 'spring'
problem = 'mi_spring'
#problem = 'mi_pressure_vessel'
#problem = 'mi_chemical_process'

#problem = 'ackley'
#problem = 'shifted_ackley'
#problem = 'dejong'
#problem = 'shifted_dejong'
#problem = 'easom'
#problem = 'shifted_easom'
#problem = 'griewank'
#problem = 'shifted_griewank'
#problem = 'rastrigin'
#problem = 'shifted_rastrigin'
#problem = 'rosenbrock'
#problem = 'shifted_rosenbrock'

#problem = 'tsp'    
#        tsp_prob=dcs.Read_TSP('TSPLIB\eil51.tsp')
#        S.of=426.0
#        tsp_prob=dcs.Read_TSP('TSPLIB\st70.tsp')
#        S.of=675.0
#        tsp_prob=zdcs.Read_TSP('TSPLIB\pr107.tsp')
#        S.of=44303.0
#        tsp_prob=zdcs.Read_TSP('TSPLIB\\bier127.tsp')
#        S.of=118282.0
#        tsp_prob=zdcs.Read_TSP('TSPLIB\\ch150.tsp')
#        S.of=6528.0
#        (timeline)=zdcs.DCS(opt_funct,S,tsp_prob)

# Initialize variables
gh = GnoweeHeuristics()
maxIter = 100      # Number of algorithm iterations
evalInterval = 500  # Fuction eval interval at which the fitness is sampled.  
history = []        # List that contains the final timeline results from each optimization run 

# History of fitness vs function evals [Feval, Fit avg, Fit std, counter]
fevalHistory = np.array([[i*evalInterval,0.0,0.0,0] for i in \
                            range(int(gh.maxFevals/evalInterval)+1)])

# Run optimization
for i in range(0,maxIter,1):
    print "Run #{}".format(i)

    # Set optimization settings
    gh.set_preset_params(problem, 'Gnowee')
    gh.initSampling = 'lhc'
    gh.population = 25
    gh.alpha = 0.5
    gh.fracDiscovered = 0.2
    gh.fracElite = 0.2
    gh.fracLevy = 0.2
    gh.stallLimit = 225
    gh.scalingFactor = 10
    gh.maxFevals = 200000
    gh.penalty = 0

    (timeline) = Gnowee.main(gh)
    
    # Save final timeline data for future processing
    minGen = min(timeline, key=attrgetter('fitness'))
    history.append(Event(minGen.generation,minGen.evaluations,minGen.fitness,
                             minGen.design))

    # Update Fitness vs Feval History
    k = 1 
    
    # Compute the average and standard deviation using a recurrence relation
    for j in range(0, len(fevalHistory),1):
        while timeline[k].evaluations < fevalHistory[j,0] and k+1 < len(timeline):  
            k += 1
        if k+1 == len(timeline) and timeline[k].evaluations < fevalHistory[j, 0]:
            # Initialize the array on the first run
            if fevalHistory[j, 3] == 0:
                fevalHistory[j, 1] = timeline[k].fitness
                fevalHistory[j, 2] = 0.0
                fevalHistory[j, 3] += 1
            else:
                oldMean = fevalHistory[j,1]
                fevalHistory[j, 3] += 1
                fevalHistory[j, 1]=fevalHistory[j, 1]\
                                 +(timeline[k].fitness-fevalHistory[j,1])/fevalHistory[j,3]
                fevalHistory[j, 2] = fevalHistory[j, 2] \
                                 +(timeline[k].fitness-oldMean)*(timeline[k].fitness \
                                                                 -fevalHistory[j,1])                
            break
        else:
            # Initialize the array on the first run
            if fevalHistory[j, 3] == 0:
                fevalHistory[j, 1] = timeline[k-1].fitness
                fevalHistory[j, 2] = 0.0
                fevalHistory[j, 3] += 1
            else:    
                oldMean = fevalHistory[j,1]
                fevalHistory[j, 3] += 1
                fevalHistory[j, 1] = fevalHistory[j,1]+(timeline[k-1].fitness \
                                     -fevalHistory[j,1])/fevalHistory[j,3]
                fevalHistory[j, 2] = fevalHistory[j,2]+(timeline[k-1].fitness \
                                     -oldMean)*(timeline[k-1].fitness-fevalHistory[j,1])
            
    #op.Plot_Feval_Hist(data=fevalHistory)
    
# Calculate averages and standard deviations
tmp = []
for i in range(len(history[-1].design)):
    if gh.objective.func.__name__ != "TSP_Obj":
        tmp.append(sum(c.design[i] for c in history)/float(len(history)))
averages = Event(sum(c.generation for c in history)/float(len(history)),
                     sum(c.evaluations for c in history)/float(len(history)),
                     sum(c.fitness for c in history)/float(len(history)),tmp)

tmp=[]
if maxIter > 1:
    for i in range(len(history[-1].design)):
        if gh.objective.func.__name__ != "TSP_Obj":
            tmp.append(m.sqrt(sum([(c.design[i]-averages.design[i])**2 for c in history]) \
                              /(len(history) - 1)))
    stdDev = Event(m.sqrt(sum([(c.generation-averages.generation)**2 for c in history]) \
                               /(len(history) - 1)),
                        m.sqrt(sum([(c.evaluations- averages.evaluations)**2 for c in history]) \
                               /(len(history) - 1)),
                        m.sqrt(sum([(c.fitness - averages.fitness)**2 for c in history]) \
                               /(len(history) - 1)),tmp)
else:
    tmp = np.zeros(len(history[-1].design))
    stdDev = Event(0, 0, 0.0, tmp)

# Trim empty feval histories 
if fevalHistory[-1,3 ] == 0:
    fevalHistory = np.array([fevalHistory[tmp, :] for tmp in \
                            range(len(fevalHistory[0:(np.argmin(fevalHistory[:, 3]))]))])

# Compute relative error (%Diff) for feval histories
if gh.optimum == 0.0:
    fevalHistory[:, 1] = fevalHistory[:, 1]*100
else:
    fevalHistory[:, 1] = (fevalHistory[:, 1]-gh.optimum)/gh.optimum*100
    fevalHistory[:, 1] = [cur if i >= 0 else 0 for cur in fevalHistory[:, 1]]

# Compute standard deviation for feval histories
for i in range(0, len(fevalHistory)):
    if fevalHistory[i, 2] != 0.0:
        fevalHistory[i, 2] = np.sqrt(fevalHistory[i, 2]/(fevalHistory[i, 3]-1))*100

# Output average results to the user
print "\nThe Average Optimized Solution for {}:".format(gh.objective.func.__name__)
print "================================"
print "Design:"
for i in range(len(averages.design)):
    print "   var %d: %4.6f $\mypm$ %4.5f " %(i+1, averages.design[i], stdDev.design[i])
print "Fitness: %4.6f $\mypm$ %4.5f " %(averages.fitness, stdDev.fitness)
print "Funct Evals: %d $\mypm$ %d " %(averages.evaluations, stdDev.evaluations)
print "Generations: %3.1f $\mypm$ %3.1f "  %(averages.generation, stdDev.generation)
if gh.optimum == 0.0:
    print "The performance metric is %4.1f" %(averages.fitness*(averages.evaluations \
                                                                +3*stdDev.evaluations))
else:
    print "The performance metric is %4.1f" %(abs((averages.fitness-gh.optimum) \
                                                  /gh.optimum) \
                                              *(averages.evaluations+3*stdDev.evaluations))

#Determine the best values obtained
best = min(history, key=attrgetter('fitness'))

# Output best result to the user
print "\nThe Best Optimized Solution for {}:".format(gh.objective.func.__name__)
print "================================"
print "Design:"
for i in range(len(averages.design)):
    print "   var %d: %4.6f " %(i+1,best.design[i])
print "Fitness: %4.6f " %best.fitness
print "Funct Evals: %d " %best.evaluations
print "Generations: %d "  %best.generation
if gh.optimum == 0.0:
    print "The performance metric is %4.1f" %(best.fitness*best.evaluations)
else:
    print "The performance metric is %4.1f" %(abs((best.fitness-gh.optimum) \
                                                  /gh.optimum)*best.evaluations)

#Plot the optimization process
if  gh.objective.func.__name__ != "TSP_Obj":
    op.plot_vars(timeline, lowBounds=gh.lb, upBounds=gh.ub, title=gh.pltTitle, label=gh.varNames)
fevals=[tmp.evaluations for tmp in history]
if maxIter >1:
    op.plot_hist(fevals,title=gh.histTitle)
op.plot_feval_hist(data=fevalHistory)

In [ ]:
print 'np.array([',
for i in range(0,len(fevalHistory)-1,1):
    print '[',fevalHistory[i,0],',',fevalHistory[i,1],',',fevalHistory[i,2],',',fevalHistory[i,3],'],',
print '[',fevalHistory[-1,0],',',fevalHistory[-1,1],',',fevalHistory[-1,2],',',fevalHistory[-1,3],']])'

In [ ]:
print np.asarray(fevals)
fevals_sa=np.array([5238,1559,2817,3526,3416,4152,2206,4245,3156,10541,4180,8432,8730,788,21581,33875,181,7497,10765,10356,13892,1464,11292,852,3102,14327,11751,10614,6644,5555,10032,11013,497,2561,12933,11375,518,16274,5350,548,11229,7942,2734,14784,2931,900,2321,5614,3524,4352,519,440,4724,2815,185,12976,1729,3880,3837,5816,8181,235,8843,464,516,5871,776,3091,1487,8701,2428,13321,1416,450,10528,10877,3704,15275,13844,1482,2233,12924,22809,1886,17178,3604,5780,7805,16553,16035,8262,2168,11495,4708,29899,270,3082,13516,6173,2082,3228,26509,10969,7620,7338,19709,3391,20966,3466,1664,5640,146,18900,1051,4302,12719,5008,1230,20339,4284,1077,10217,3022,2079,13399,8788,15191,8916,18181,2206,8424,13546,1853,6575,2497,10146,3416,3752,1215,617,4827,7514,1542,15413,623,13647,4351,8758,4714,565,8811,4506,1104,3203,910,2589,7751,853,13087,2690,11162,4630,11260,4641,2950,1806,1245,9654,218,5882,9724,4935,11808,3156,1541,240,6712,1495,8407,16436,11959,4825,1488,3310,5670,13924,14795,5077,7557,278,8377,1151,3725,23145,1540,19840,3423,7192,182,15090,1752,9963,27604,2140,2528,5210,11251,10318,14631,727,849,5221,1186,1777,5951,3280,9678,14095,16917,5966,7607,508,9909,1830,11126,22264,11875,12423,10171,467,1443,7163,1418,7074,2047,12614,13642,1480,467,13035,1808,4272,2849,1486,7382,2904,187,3764,8881,12350,8842,11332,13676,7266,7049,1100,4946,1259,2081,177,266,12580,7719,10423,1743,21977,11220,3780,3282,6753,7432,3801,845,3454,9555,4033,3042,2571,860,189,822,4179,12620,2740,8534,1933,6011,7410,4081,2548,12448,3761,1207,2242,16559,180,123,6490,21519,187,5040,1075,14680,2408,14243,11285,2096,801,7385,20283,1804,5565,168,177,1169,3047,10511,6920,11486,9818,1341,1224,6253,2155,8620,1482,16551,13096,1487,2894,10638,2610,1108,11471,6242,8306,18901,2745,235,11186,10598,6738,2463,1171,1158,3531,2557,20031,6640,1509,928,1072,1504,1416,8688,13121,11779,2819,4109,4981,12213,4446,9152,2518,3381,801,3973,9006,881,1490,4515,2453,2849,2799,6137,5850,387,8509,7797,9408,3129,1961,14542,16877,4387,1507,14145,14791,13986,1176,1523,10289,10404,10555,5804,14445,1843,611,2486,12849,1406,1424,1296,1354,11575,15491,6910,644,11406,201,15140,5246,12448,2117,4998,14934,8828,7058,3394,3100,10526,2932,2987,771,4281,4529,548,159,1763,1829,570,9929,6003,2602,1743,11276,1495,8218,2241,575,5392,3120,630,16880,13480,4797,3471,6937,4705,4550,12185,5786,3093,5426,5602,2783,2244,9728,17860,6228,4651,1162,478,6152,446,5676,2812,1727,2526,680,1175,2621,15396,7618,258,26964,205,6425,9067,3044,7175,567,8013,15140,10895,2389,5945,8057,1505,609,10301,183,9313,3649,19532,1580,12922,17984,8191,5101,10933,17647,2377,4775,15259,5170,9459,7996,4065,6290,3250,18144,7907,2439,1143,1194,5679,11949,1435,4723,24608,632,2588,1117,4014,6137,2011,2923,10183,1403,3766,4423,11525,8303,10472,11815,3106,17964,1247,2733,514,16436,8255,9979,495,10189,5637,8987,204,2510,9937,580,12268,14791,5330,228,3419,8556,11264,10912,16607,10947,10058,1165,13114,4435,2197,5680,4489,2068,504,2101,1484,3925,2251,1466,2114,6038,3517,1769,9388,22773,2037,4617,1689,1716,2406,181,3754,215,1125,15972,478,3321,13025,6010,2695,13233,4255,10434,3526,17869,748,4194,7953,4882,2401,1783,2121,1214,17426,2741,10939,10623,158,646,597,5114,13098,1463,3486,1241,4020,3263,19729,10982,834,1984,7154,2897,7695,8149,480,4840,500,13207,2809,4397,1158,12157,806,610,15135,397,2840,8636,1832,11215,1750,2916,8365,1524,2037,1543,1368,4420,7806,1467,3056,5773,4190,11963,848,1732,13152,15071,10848,12540,489,3179,11950,9816,1442,876,16878,3232,18647,3024,11423,1170,4044,1281,1717,3704,8525,7689,1632,10906,10947,1553,842,3410,19792,10223,10428,484,588,10499,4122,2483,2685,201,3430,8908,11617,782,5436,13070,7250,4390,15560,793,6455,13012,2167,3276,5096,5469,153,425,7912,1058,12555,191,211,4592,4207,268,10123,5342,4067,18088,1416,4310,16645,578,1604,9217,3514,13589,422,23209,6281,14466,2467,1101,3692,6371,18763,11239,3663,5406,491,3102,9377,11083,1215,223,213,203,472,6608,12531,14478,1625,1817,5587,480,6751,8796,837,1897,3833,5682,862,14714,3517,12947,469,15108,1057,859,5091,1212,1292,4126,11877,2040,20828,3469,2996,3679,6173,854,13293,2204,1882,396,1804,15969,2817,3770,603,5480,1105,2734,1978,2116,3566,834,3028,1612,9141,3403,1148,1494,2455,8944,2903,798,8604,1566,931,1209,4592,997,4891,1521,1929,853,7472,917,1405,812,10748,12374,9579,16117,150,1810,1845,515,208,5777,489,1530,4198,14846,2384,9130,2261,18337,1242,6522,3135,4007,811,829,5234,10452,3166,9221,3815,14020,9217,12777,267,3346,492,10482,3022,3599,7846,1114,1489,3825,1175,1378,5883,11869,12792,3466,2922,6499,8404,7497,17463,2560,10923,977,11696,4245,466,5411,12930,10598,5539,865,3881,855,3841,16362,3456,1497,5797,6821,802,894,2175,5318,2877,3614,14600,2038,3792,8653,189,18744,12735,8257,1597,18335,2827,3571,10583,137,2310,9460,3555,510,11058,2565,13065,6327,11542,2504,2734,3725,6767,22992,11720,4595,10179,3682,6393,4064,1485,11309,4344,769,19731,6463,5793,10974,1804,6870,479,7142,1760,10361,3408,3680,12037,8408,10443,1562,21544,528,13234,2703,10900,4909,3720,7411,3525,6944,8929,6152,8100,10009,15069,9047,1504,20073,2920,815,17845,2083,499,2512,9701,13120,10500,6136,13150,7606,4330,3294,12156,12576])
print fevals_sa